Check npy format

In [ ]:
import numpy as np
file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/mips/X_mips.npy"
data = np.load(file_path)

print(data.shape)

print(data[:5])

In [ ]:
mips_data = np.load("/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/mips/X_x86.npy")
print(mips_data.shape)

In [ ]:
label = np.load("/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/mips/y.npy")

print(label.shape)
print(label[:10])
print(label[40000:40010])

In [ ]:
file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/X_arm_merged.npy"
data = np.load(file_path)
print(data.shape)

In [ ]:
import pickle 

file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/word2vec/CBOW/sentences_train.pkl"

with open(file_path, 'rb') as f:
    sentences = pickle.load(f)

print(len(sentences))

In [ ]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_token/train_arm_token_mixed.pickle"

#check pickle filename and function name
with open(file_path, 'rb') as f:
    data = pickle.load(f)

#Check index 0-10 and 28748-28758
data2 = data[0:10]
data3 = data[28748:28758]

print(data2)
print(data3)

Check npy similarity

In [ ]:
import torch
import numpy as np

def similarity_score(x_arm, x_mapped):
    # Eq.(7): s = 1 / (1 + ||arm - mapped||_2)
    dist = torch.norm(x_arm - x_mapped, p=2, dim=1)  # [B]
    return 1.0 / (1.0 + dist)


arm_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/X_arm.npy"
x86_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/X_x86.npy"
label_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/y.npy"

arm_np = np.load(arm_path)
x86_np = np.load(x86_path)
label_np = np.load(label_path)

#check npy contents

print("ARM shape:", arm_np.shape)
print("x86 shape:", x86_np.shape)
print("Label shape:", label_np.shape)


print("ARM first 5 vectors:", arm_np[:5])
print("x86 first 5 vectors:", x86_np[:5])
print("Labels first 10:", label_np[:10])



In [ ]:
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# 路徑
arm_csv  = "/home/tommy/Projects/pcodeFcg/dataset/alignment/arm_32_alignment.csv"
mips_csv = "/home/tommy/Projects/pcodeFcg/dataset/alignment/mips_32_alignment.csv"
w2v_path  = "/home/tommy/Projects/pcodeFcg/vector/contrastive/word2vec/CBOW/word2vec.model"

# 讀檔
df_arm  = pd.read_csv(arm_csv)   # 欄位：file_name,function_name,pcode,label
df_mips = pd.read_csv(mips_csv)
w2v     = Word2Vec.load(w2v_path)

# 前 5 筆同列比對（ARM vs MIPS row-wise）
print("=== 同一列前 5 筆 Cosine Similarity ===")
for i in range(5):
    toks_arm  = str(df_arm.loc[i, "pcode"]).split()
    toks_mips = str(df_mips.loc[i, "pcode"]).split()
    vec_arm   = np.mean([w2v.wv[t] for t in toks_arm  if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    vec_mips  = np.mean([w2v.wv[t] for t in toks_mips if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    sim        = cosine_similarity(vec_arm.reshape(1,-1), vec_mips.reshape(1,-1))[0,0]
    print(f"Row {i}: {sim:.4f}")

# 隨機 5 筆不同行比對（ARM row i vs MIPS row j, i != j）
print("\n=== 隨機 5 筆不同行 Cosine Similarity ===")
n = len(df_arm)
pairs = set()
while len(pairs) < 5:
    i = random.randrange(n)
    j = random.randrange(n)
    if i != j:
        pairs.add((i, j))

for i, j in pairs:
    toks_arm  = str(df_arm.loc[i, "pcode"]).split()
    toks_mips = str(df_mips.loc[j, "pcode"]).split()
    vec_arm   = np.mean([w2v.wv[t] for t in toks_arm  if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    vec_mips  = np.mean([w2v.wv[t] for t in toks_mips if t in w2v.wv] or [np.zeros(w2v.vector_size)], axis=0)
    sim        = cosine_similarity(vec_arm.reshape(1,-1), vec_mips.reshape(1,-1))[0,0]
    print(f"ARM row {i} vs MIPS row {j}: {sim:.4f}")


In [ ]:
import os
import pickle
import torch
import networkx as nx

# 1. 載入你訓練好的 Adapter
from model import Adapter  
ckpt = torch.load(
    '/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/simsiam_adapter.pth',
    map_location='cpu'
)
state_dict = ckpt.get('state_dict', ckpt)
adapter_state = {
    k[len('backbone.'):]: v
    for k, v in state_dict.items()
    if k.startswith('backbone.adapter.')
}
adapter = Adapter(dim=256, num_blocks=2)
adapter.load_state_dict(adapter_state, strict=False)
adapter.eval()

# 2. 指定單一對應檔案路徑
path_src = '/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/train/0b/0b61ed19ec8d9450268020524aae18ff68071164042097c346afe9085b72135d.gpickle'
path_tgt = '/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/test/0a/0a3eab21fff9e3edbb22861e6d1c461dee66e36cc2f0010602eaff0496ed8271.gpickle'

# 3. 讀入圖
with open(path_src, 'rb') as f:
    G_src = pickle.load(f)
    print(G_src)
    for node in G_src.nodes():
        print(G_src.nodes[node]['vector'])
with open(path_tgt, 'rb') as f:
    G_tgt = pickle.load(f)
    print(G_tgt)

# 4. 比較每個 node
print(f'Comparing {os.path.basename(path_src)} → {os.path.basename(path_tgt)}\n')
print('Node\tOrigDist\tMappedDist')
for node in G_src.nodes():
    if node not in G_tgt:
        continue
    x_src = torch.tensor(G_src.nodes[node]['vector'], dtype=torch.float32)
    x_tgt = torch.tensor(G_tgt.nodes[node]['vector'], dtype=torch.float32)
    # 原始距離
    d_orig = torch.dist(x_src, x_tgt).item()
    # 經 adapter 映射後距離
    x_mapped = adapter(x_src.unsqueeze(0)).squeeze(0)
    d_mapped = torch.dist(x_mapped, x_tgt).item()
    print(f'{node}\t{d_orig:.4f}\t{d_mapped:.4f}')


In [ ]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment/train.pickle"

with open(file_path, 'rb') as f:
    data = pickle.load(f)

print(f"Data loaded from {file_path}, length: {len(data)}")
print(data[:1])  # Print first 5 items to verify structure


In [ ]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_no_mix/test/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

with open(file_path, "rb") as fp:
    G = pickle.load(fp)
    for addr, data in G.nodes(data=True):
        print(f"Node: {addr}, Data: {data}")
        
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Shape of node vectors:{len(data['vector']) if 'vector' in data else 'N/A'}")
    print(f"Sample node data: {list(G.nodes(data=True))[1]}")



In [ ]:
file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/word2vec/CBOW_no_mix/sentences_train.pkl"
with open(file_path, 'rb') as f:
    sentences = pickle.load(f)
print(f"Number of sentences: {len(sentences)}")
print(f"First 5 sentences: {sentences[:5]}")

In [ ]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_token/train_arm_token.pickle"

with open(file_path, 'rb') as f:
    data = pickle.load(f)

print(type(data))

for i, item in enumerate(data):
    print(f"Item {i}: {item}")

In [ ]:
import pickle
file_path="/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

with open(file_path, 'rb') as fp:
    G = pickle.load(fp)
    print(G)
first_node = list(G.nodes(data=True))[0]
# print(first_node)

vector = first_node[1]['vector']
print(f"Vector length: {len(vector)}")

In [ ]:
from model import Adapter
import torch
model_path = '/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/simsiam_adapter.pth'
ckpt = torch.load(model_path, map_location='cpu')
state_dict = ckpt.get('state_dict', ckpt)
adapter_state = {
    k[len('backbone.'):]: v
    for k, v in state_dict.items()
    if k.startswith('backbone.adapter.')
}

print(list(adapter_state.keys()))


adapter = Adapter(dim=256, num_blocks=2)
missing, unexpected = adapter.load_state_dict(adapter_state, strict=False)
print('Missing keys:', missing)
print('Unexpected keys:', unexpected)
adapter.eval()


In [ ]:
import pickle
import torch
import networkx as nx

def load_vectors(path):
    with open(path, "rb") as f:
        G = pickle.load(f) 
    return [torch.tensor(d['vector'], dtype=torch.float32) for _, d in G.nodes(data=True)]

def avg_cosine_similarity(vecs1, vecs2):
    mean1 = torch.stack(vecs1).mean(dim=0)
    mean2 = torch.stack(vecs2).mean(dim=0)
    cos_sim = torch.nn.functional.cosine_similarity(mean1.unsqueeze(0), mean2.unsqueeze(0))
    return cos_sim.item()

# file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train/00/0036f76612926fba2f71297dd4d13f326173f38662e8f639a5f1db0195cc56ee.gpickle"
# arm_file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train_transfer/00/0036f76612926fba2f71297dd4d13f326173f38662e8f639a5f1db0195cc56ee.gpickle"
arm_file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_transfer/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

vecs1 = load_vectors(file_path)
vecs2 = load_vectors(arm_file_path)

score = avg_cosine_similarity(vecs1, vecs2)
print(f"Average cosine similarity: {score:.4f}")


In [ ]:
import os
import random
import pickle
import torch
import pandas as pd

def load_vectors(path):
    with open(path, "rb") as f:
        G = pickle.load(f)
    return [torch.tensor(d['vector'], dtype=torch.float32) for _, d in G.nodes(data=True)]

def avg_file_vector(path):
    vecs = load_vectors(path)
    return torch.stack(vecs).mean(dim=0)

def cosine_similarity(v1, v2):
    return torch.nn.functional.cosine_similarity(
        v1.unsqueeze(0), v2.unsqueeze(0)
    ).item()

# paths
train_csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/train.csv"
test_csv_path  = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/test_arm.csv"

train_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train"
test_dir  = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm"
train_transfer_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train_transfer_merged"
test_transfer_dir  = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_transfer_merged"

# load CSVs
train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

# group by labels
labels = ["benign", "mirai", "gafgyt", "tsunami"]

results = {}

for label in labels:
    # 找到該 label 的檔案
    train_files = train_df[train_df["label"] == label]["file_name"].tolist()
    test_files  = test_df[test_df["label"] == label]["file_name"].tolist()

    if not train_files or not test_files:
        continue

    # 隨機取最多 100 個 (不足就全取)
    train_files = random.sample(train_files, min(100, len(train_files)))
    test_files  = random.sample(test_files, min(100, len(test_files)))

    sims_before, sims_after = [], []

    for tf in train_files:
        train_path = os.path.join(train_dir, tf + ".gpickle")
        train_path_t = os.path.join(train_transfer_dir, tf + ".gpickle")
        if not os.path.exists(train_path) or not os.path.exists(train_path_t):
            continue
        v_train = avg_file_vector(train_path)
        v_train_t = avg_file_vector(train_path_t)

        for tef in test_files:
            test_path = os.path.join(test_dir, tef + ".gpickle")
            test_path_t = os.path.join(test_transfer_dir, tef + ".gpickle")
            if not os.path.exists(test_path) or not os.path.exists(test_path_t):
                continue
            v_test = avg_file_vector(test_path)
            v_test_t = avg_file_vector(test_path_t)

            sims_before.append(cosine_similarity(v_train, v_test))
            sims_after.append(cosine_similarity(v_train_t, v_test_t))

    if sims_before and sims_after:
        results[label] = {
            "before": sum(sims_before) / len(sims_before),
            "after": sum(sims_after) / len(sims_after),
        }

# 印出結果
for label, vals in results.items():
    print(f"Label: {label}")
    print(f"  Avg similarity before transfer: {vals['before']:.4f}")
    print(f"  Avg similarity after  transfer: {vals['after']:.4f}")


In [ ]:
import os
import random
import pickle
import torch

def load_vectors(path):
    with open(path, "rb") as f:
        G = pickle.load(f)
    return [torch.tensor(d['vector'], dtype=torch.float32) for _, d in G.nodes(data=True)]

def avg_cosine_similarity(vecs1, vecs2):
    mean1 = torch.stack(vecs1).mean(dim=0)
    mean2 = torch.stack(vecs2).mean(dim=0)
    cos_sim = torch.nn.functional.cosine_similarity(mean1.unsqueeze(0), mean2.unsqueeze(0))
    return cos_sim.item()

# 原始 & transfer 資料夾
train_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train"
test_dir  = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test"
train_transfer_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train_transfer_O0"
test_transfer_dir  = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_transfer_O0"

# 收集檔案
train_files, test_files = [], []
for root, _, files in os.walk(train_dir):
    for f in files:
        if f.endswith(".gpickle"):
            train_files.append(os.path.join(root, f))
for root, _, files in os.walk(test_dir):
    for f in files:
        if f.endswith(".gpickle"):
            test_files.append(os.path.join(root, f))

# 隨機抽 50 對
pairs = random.sample(list(zip(train_files, test_files)), 1000)

sims_before, sims_after = [], []

for train_f, test_f in pairs:
    rel_train = os.path.relpath(train_f, train_dir)
    rel_test  = os.path.relpath(test_f, test_dir)
    train_f_transfer = os.path.join(train_transfer_dir, rel_train)
    test_f_transfer  = os.path.join(test_transfer_dir, rel_test)

    vecs1 = load_vectors(train_f)
    vecs2 = load_vectors(test_f)
    sims_before.append(avg_cosine_similarity(vecs1, vecs2))

    vecs1_t = load_vectors(train_f_transfer)
    vecs2_t = load_vectors(test_f_transfer)
    sims_after.append(avg_cosine_similarity(vecs1_t, vecs2_t))

print(f"Average similarity before transfer: {sum(sims_before)/len(sims_before):.4f}")
print(f"Average similarity after  transfer: {sum(sims_after)/len(sims_after):.4f}")


Check pickle

In [ ]:
import pickle 

with open("/home/tommy/Projects/pcodeFcg/dataset/alignment_token/train_arm_token_Os.pickle", 'rb') as f:
    data = pickle.load(f)
print(len(data))
print(data[:5])

In [ ]:
import numpy as np
import torch

# 檔案路徑
file_path_arm = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/X_arm_merged.npy"
file_path_x86 = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/X_x86_merged.npy"
file_path_y   = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/y_merged.npy"

# 讀取 numpy 向量
X_arm = np.load(file_path_arm)
X_x86 = np.load(file_path_x86)
y     = np.load(file_path_y)

# 轉成 torch tensor
X_arm_t = torch.tensor(X_arm, dtype=torch.float32)
X_x86_t = torch.tensor(X_x86, dtype=torch.float32)

# 計算平均 cosine similarity
def avg_cosine_similarity(vecs1, vecs2):
    mean1 = vecs1.mean(dim=0, keepdim=True)
    mean2 = vecs2.mean(dim=0, keepdim=True)
    cos_sim = torch.nn.functional.cosine_similarity(mean1, mean2)
    return cos_sim.item()

cos_sim = avg_cosine_similarity(X_arm_t, X_x86_t)
print(f"ARM vs x86 平均 cosine similarity = {cos_sim:.4f}")


In [ ]:
import pickle
import torch
from model import Adapter
import numpy as np

alignment_model = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/simsiam_adapter_merged.pth"
file_path_arm = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/X_arm_merged.npy"
file_path_x86 = "/home/tommy/Projects/pcodeFcg/vector/contrastive/resnet/arm_cbow_v2/X_x86_merged.npy"

X_arm = np.load(file_path_arm)
X_x86 = np.load(file_path_x86)
X_arm_t = torch.tensor(X_arm, dtype=torch.float32)
X_x86_t = torch.tensor(X_x86, dtype=torch.float32)

ckpt = torch.load(alignment_model, map_location='cpu')
state_dict = ckpt.get('state_dict', ckpt)
adapter_state = {
    k[len('backbone.'):]: v
    for k, v in state_dict.items()
    if k.startswith('backbone.adapter.')
}
adapter = Adapter(dim=256, num_blocks=2)
adapter.load_state_dict(adapter_state, strict=False)
adapter.eval()

with torch.no_grad():
    z_arm = adapter(X_arm_t) 
    z_x86 = adapter(X_x86_t) 

# 計算平均 cosine similarity
def avg_cosine_similarity(vecs1, vecs2):
    mean1 = vecs1.mean(dim=0, keepdim=True)
    mean2 = vecs2.mean(dim=0, keepdim=True)
    cos_sim = torch.nn.functional.cosine_similarity(mean1, mean2)
    return cos_sim.item()

cos_before = avg_cosine_similarity(X_arm_t, X_x86_t)
cos_after  = avg_cosine_similarity(z_arm, z_x86)

print(f"對齊前 ARM vs x86 平均 cosine similarity = {cos_before:.4f}")
print(f"對齊後 ARM vs x86 平均 cosine similarity = {cos_after:.4f}")


In [ ]:
import os
import csv
import pickle
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/train.csv"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_4_class/train"

df = pd.read_csv(csv_path)  # file_name, CPU, label

file_vectors = []
labels = []
cpus = []

for _, row in df.iterrows():
    file_name = row["file_name"]
    cpu = row["CPU"]
    label = row["label"]
    prefix = file_name[:2]
    gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")
    if not os.path.exists(gpickle_path):
        print(f"[WARN] gpickle not found: {gpickle_path}")
        continue
    with open(gpickle_path, "rb") as f:
        G = pickle.load(f)
    node_vecs = []
    for _, d in G.nodes(data=True):
        if "vector" in d:
            node_vecs.append(torch.tensor(d["vector"], dtype=torch.float32))

    if not node_vecs:
        continue

    mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
    file_vectors.append(mean_vec)
    labels.append(label)
    cpus.append(cpu)

file_vectors = np.array(file_vectors)

le = LabelEncoder()
label_ids = le.fit_transform(labels) 

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 畫圖
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    tsne_result[:, 0], tsne_result[:, 1],
    c=label_ids, cmap="tab10", alpha=0.7
)
handles, _ = scatter.legend_elements()
plt.legend(handles, le.classes_, title="Label")

plt.title("t-SNE of gpickle file representations")
plt.show()


In [ ]:
import os
import csv
import pickle
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/train.csv"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train"

df = pd.read_csv(csv_path)  # file_name, CPU, label

file_vectors = []
labels = []
cpus = []

for _, row in df.iterrows():
    file_name = row["file_name"]
    cpu = row["CPU"]
    label = row["label"]
    prefix = file_name[:2]
    gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")
    if not os.path.exists(gpickle_path):
        print(f"[WARN] gpickle not found: {gpickle_path}")
        continue
    with open(gpickle_path, "rb") as f:
        G = pickle.load(f)
    node_vecs = []
    for _, d in G.nodes(data=True):
        if "vector" in d:
            node_vecs.append(torch.tensor(d["vector"], dtype=torch.float32))

    if not node_vecs:
        continue

    mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
    file_vectors.append(mean_vec)
    labels.append(label)
    cpus.append(cpu)

file_vectors = np.array(file_vectors)

le = LabelEncoder()
label_ids = le.fit_transform(labels) 

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 畫圖
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    tsne_result[:, 0], tsne_result[:, 1],
    c=label_ids, cmap="tab10", alpha=0.7
)
handles, _ = scatter.legend_elements()
plt.legend(handles, le.classes_, title="Label")

plt.title("t-SNE of gpickle file representations")
plt.show()


In [ ]:
import os
import csv
import pickle
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/train.csv"
csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/test_arm.csv"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train_transfer"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transfer"

df = pd.read_csv(csv_path)  # file_name, CPU, label

file_vectors = []
labels = []
cpus = []

for _, row in df.iterrows():
    file_name = row["file_name"]
    cpu = row["CPU"]
    label = row["label"]
    prefix = file_name[:2]
    gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")
    if not os.path.exists(gpickle_path):
        print(f"[WARN] gpickle not found: {gpickle_path}")
        continue
    with open(gpickle_path, "rb") as f:
        G = pickle.load(f)
    node_vecs = []
    for _, d in G.nodes(data=True):
        if "vector" in d:
            node_vecs.append(torch.tensor(d["vector"], dtype=torch.float32))

    if not node_vecs:
        continue

    mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
    file_vectors.append(mean_vec)
    labels.append(label)
    cpus.append(cpu)

file_vectors = np.array(file_vectors)

le = LabelEncoder()
label_ids = le.fit_transform(labels) 

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 畫圖
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    tsne_result[:, 0], tsne_result[:, 1],
    c=label_ids, cmap="tab10", alpha=0.7
)
handles, _ = scatter.legend_elements()
plt.legend(handles, le.classes_, title="Label")

plt.title("t-SNE of gpickle file representations")
plt.show()


In [ ]:
import os
import pickle
import torch
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 兩組資料
datasets = [
    {
        "csv": "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/train.csv",
        "gpickle": "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train_transferred_Os"
    },
    {
        "csv": "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/test_arm.csv",
        "gpickle": "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transferred_Os"
    }
]

file_vectors, labels, cpus = [], [], []

for dataset in datasets:
    df = pd.read_csv(dataset["csv"])  # file_name, CPU, label
    gpickle_dir = dataset["gpickle"]

    for _, row in df.iterrows():
        file_name = row["file_name"]
        cpu = row["CPU"]
        label = row["label"]

        prefix = file_name[:2]
        gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")

        if not os.path.exists(gpickle_path):
            print(f"[WARN] gpickle not found: {gpickle_path}")
            continue

        # G = nx.read_gpickle(gpickle_path)
        with open(gpickle_path, "rb") as f:
            G = pickle.load(f)

        node_vecs = [
            torch.tensor(d["vector"], dtype=torch.float32)
            for _, d in G.nodes(data=True) if "vector" in d
        ]

        if not node_vecs:
            continue

        mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
        file_vectors.append(mean_vec)
        labels.append(label)
        cpus.append(cpu)

file_vectors = np.array(file_vectors)

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 定義 CPU 對應顏色
cpu_colors = {
    "arm": "red",
    "mips": "blue",
    "x86": "green"
}

# 定義 label 對應 marker
label_markers = {
    "benign": "o",
    "gafgyt": "s",
    "mirai": "D",
    "tsunami": "^"
}

plt.figure(figsize=(8, 6))

for i, (x, y) in enumerate(tsne_result):
    cpu = cpus[i]
    label = labels[i]
    plt.scatter(
        x, y,
        c=cpu_colors.get(cpu, "black"),
        marker=label_markers.get(label, "o"),
        alpha=0.7,
        edgecolors="k",
        linewidths=0.3
    )

# 建立 legend
from matplotlib.lines import Line2D

legend_elements = []

# CPU legend
for cpu, color in cpu_colors.items():
    legend_elements.append(Line2D([0], [0], marker="o", color="w", label=f"CPU={cpu}",
                                  markerfacecolor=color, markersize=10))

# Label legend
for lbl, marker in label_markers.items():
    legend_elements.append(Line2D([0], [0], marker=marker, color="k", label=f"Label={lbl}",
                                  markerfacecolor="w", markersize=10))

plt.legend(handles=legend_elements, loc="best")
plt.title("t-SNE of gpickle file representations\nColor=CPU, Shape=Label")
plt.show()


In [ ]:
import os
import csv
import pickle
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/train.csv"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/train"

df = pd.read_csv(csv_path)  # file_name, CPU, label

file_vectors = []
labels = []
cpus = []

for _, row in df.iterrows():
    file_name = row["file_name"]
    cpu = row["CPU"]
    label = row["label"]
    prefix = file_name[:2]
    gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")
    if not os.path.exists(gpickle_path):
        print(f"[WARN] gpickle not found: {gpickle_path}")
        continue
    with open(gpickle_path, "rb") as f:
        G = pickle.load(f)
    node_vecs = []
    for _, d in G.nodes(data=True):
        if "vector" in d:
            node_vecs.append(torch.tensor(d["vector"], dtype=torch.float32))

    if not node_vecs:
        continue

    mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
    file_vectors.append(mean_vec)
    labels.append(label)
    cpus.append(cpu)

file_vectors = np.array(file_vectors)

le = LabelEncoder()
label_ids = le.fit_transform(labels) 

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 畫圖
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    tsne_result[:, 0], tsne_result[:, 1],
    c=label_ids, cmap="tab10", alpha=0.7
)
handles, _ = scatter.legend_elements()
plt.legend(handles, le.classes_, title="Label")

plt.title("t-SNE of gpickle file representations")
plt.show()


In [ ]:
import os
import csv
import pickle
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/test_arm.csv"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm"

df = pd.read_csv(csv_path)  # file_name, CPU, label

file_vectors = []
labels = []
cpus = []

for _, row in df.iterrows():
    file_name = row["file_name"]
    cpu = row["CPU"]
    label = row["label"]
    prefix = file_name[:2]
    gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")
    if not os.path.exists(gpickle_path):
        print(f"[WARN] gpickle not found: {gpickle_path}")
        continue
    with open(gpickle_path, "rb") as f:
        G = pickle.load(f)
    node_vecs = []
    for _, d in G.nodes(data=True):
        if "vector" in d:
            node_vecs.append(torch.tensor(d["vector"], dtype=torch.float32))

    if not node_vecs:
        continue

    mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
    file_vectors.append(mean_vec)
    labels.append(label)
    cpus.append(cpu)

file_vectors = np.array(file_vectors)

le = LabelEncoder()
label_ids = le.fit_transform(labels) 

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 畫圖
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    tsne_result[:, 0], tsne_result[:, 1],
    c=label_ids, cmap="tab10", alpha=0.7
)
handles, _ = scatter.legend_elements()
plt.legend(handles, le.classes_, title="Label")

plt.title("t-SNE of gpickle file representations")
plt.show()


In [ ]:
import os
import csv
import pickle
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/test_arm.csv"
gpickle_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transfer"

df = pd.read_csv(csv_path)  # file_name, CPU, label

file_vectors = []
labels = []
cpus = []

for _, row in df.iterrows():
    file_name = row["file_name"]
    cpu = row["CPU"]
    label = row["label"]
    prefix = file_name[:2]
    gpickle_path = os.path.join(gpickle_dir, prefix, file_name + ".gpickle")
    if not os.path.exists(gpickle_path):
        print(f"[WARN] gpickle not found: {gpickle_path}")
        continue
    with open(gpickle_path, "rb") as f:
        G = pickle.load(f)
    node_vecs = []
    for _, d in G.nodes(data=True):
        if "vector" in d:
            node_vecs.append(torch.tensor(d["vector"], dtype=torch.float32))

    if not node_vecs:
        continue

    mean_vec = torch.stack(node_vecs).mean(dim=0).numpy()
    file_vectors.append(mean_vec)
    labels.append(label)
    cpus.append(cpu)

file_vectors = np.array(file_vectors)

le = LabelEncoder()
label_ids = le.fit_transform(labels) 

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(file_vectors)

# 畫圖
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    tsne_result[:, 0], tsne_result[:, 1],
    c=label_ids, cmap="tab10", alpha=0.7
)
handles, _ = scatter.legend_elements()
plt.legend(handles, le.classes_, title="Label")

plt.title("t-SNE of gpickle file representations")
plt.show()


In [ ]:
import pickle 

with open("/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector.pickle", "rb") as f:
    data = pickle.load(f)

print(len(data))
# check data structure

print(type(data))
print(data[0])

In [ ]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"
file_path_2 = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm/13/13abb7d77381e30d323c3af1fbc7e2c5c7c91c5204fa18589139020670c4ecdf.gpickle"

file_path_transfer = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transferred_Os/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"
file_path_2_transfer = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transferred_Os/13/13abb7d77381e30d323c3af1fbc7e2c5c7c91c5204fa18589139020670c4ecdf.gpickle"

with open(file_path, "rb") as fp:
    G = pickle.load(fp)
    print(G)
    first_node = list(G.nodes(data=True))[0]
    # print(first_node)
    vector = first_node[1]['vector']
    print(f"Vector length: {len(vector)}")

with open(file_path_2, "rb") as fp:
    G = pickle.load(fp)
    print(G)
    first_node = list(G.nodes(data=True))[0]
    # print(first_node)
    vector = first_node[1]['vector']
    print(f"Vector length: {len(vector)}")

In [ ]:
import pickle
import torch
import networkx as nx

def load_avg_vector(file_path):
    with open(file_path, "rb") as fp:
        G = pickle.load(fp)
    # 把所有 node 的 vector 取平均
    vectors = [torch.tensor(d['vector'], dtype=torch.float32) for _, d in G.nodes(data=True)]
    if len(vectors) == 0:
        return None
    return torch.stack(vectors).mean(dim=0)

def cosine_similarity(v1, v2):
    return torch.nn.functional.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0)).item()

file_path = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"
file_path_2 = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm/13/13abb7d77381e30d323c3af1fbc7e2c5c7c91c5204fa18589139020670c4ecdf.gpickle"

file_path_transfer = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transferred_Os/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"
file_path_2_transfer = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2/test_arm_transferred_Os/13/13abb7d77381e30d323c3af1fbc7e2c5c7c91c5204fa18589139020670c4ecdf.gpickle"

# 原始
vec1 = load_avg_vector(file_path)
vec2 = load_avg_vector(file_path_2)
orig_sim = cosine_similarity(vec1, vec2)

# transferred 後
vec1_t = load_avg_vector(file_path_transfer)
vec2_t = load_avg_vector(file_path_2_transfer)
transfer_sim = cosine_similarity(vec1_t, vec2_t)

print(f"Original similarity:   {orig_sim:.4f}")
print(f"Transferred similarity: {transfer_sim:.4f}")


In [ ]:
import os
import pickle
import torch
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

def load_avg_vector(file_path):
    if not os.path.exists(file_path):
        return None
    try:
        with open(file_path, "rb") as fp:
            G = pickle.load(fp)
        vectors = [torch.tensor(d['vector'], dtype=torch.float32) for _, d in G.nodes(data=True)]
        if not vectors:
            return None
        return torch.stack(vectors).mean(dim=0)
    except Exception:
        return None

def cosine_similarity(v1, v2):
    return torch.nn.functional.cosine_similarity(
        v1.unsqueeze(0), v2.unsqueeze(0)
    ).item()

def process_pair(row):
    file_name = row["file_name"]
    base_dir = "/home/tommy/Projects/pcodeFcg/vector/contrastive/GNN/arm_cbow_v2"

    orig_path = os.path.join(base_dir, "test_arm", file_name[:2], file_name + ".gpickle")
    trans_path = os.path.join(base_dir, "test_transferred_align_labse", file_name[:2], file_name + ".gpickle")

    v_orig = load_avg_vector(orig_path)
    v_trans = load_avg_vector(trans_path)

    if v_orig is None or v_trans is None:
        return None

    return {"file": file_name, "orig_vector": v_orig, "trans_vector": v_trans}

if __name__ == "__main__":
    csv_path = "/home/tommy/Projects/pcodeFcg/dataset/csv/temp/test_arm.csv"
    df = pd.read_csv(csv_path)
    df = df[df["label"] == "benign"]  # 只跑 benign

    rows = df.to_dict(orient="records")

    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(pool.imap(process_pair, rows), total=len(rows)))

    results = [r for r in results if r]  # 過濾掉缺失或錯誤

    # 隨機選 100 對 pair 減少 O(n^2) 計算量
    import random
    sims_orig, sims_trans = [], []
    sample_pairs = random.sample(
        [(i, j) for i in range(len(results)) for j in range(i + 1, len(results))],
        min(100, len(results) * (len(results) - 1) // 2)  # 最多 100 對
    )

    for i, j in tqdm(sample_pairs, desc="Computing similarities"):
        sims_orig.append(cosine_similarity(results[i]["orig_vector"], results[j]["orig_vector"]))
        sims_trans.append(cosine_similarity(results[i]["trans_vector"], results[j]["trans_vector"]))

    print(f"\n平均 Original similarity:   {sum(sims_orig) / len(sims_orig):.4f}")
    print(f"平均 Transferred similarity: {sum(sims_trans) / len(sims_trans):.4f}")
    print(f"有效檔案數: {len(results)} / {len(rows)}")


Computing similarities: 100%|██████████| 100/100 [00:00<00:00, 5382.14it/s]


平均 Original similarity:   0.9309
平均 Transferred similarity: 0.4702
有效檔案數: 441 / 450
